<a href="https://colab.research.google.com/github/Jomucon21muri/Aprendizaje_automatico/blob/main/01_Sistemas_aprendizaje_automatico/01_Ml_supervisado/ml_supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Aprendizaje Automático Supervisado: Fundamentos y Aplicaciones

---

## Resumen

El **aprendizaje supervisado** constituye el paradigma fundamental del aprendizaje automático moderno, caracterizado por la utilización de conjuntos de datos **etiquetados** para entrenar modelos predictivos.

En este notebook exploraremos:

- 🎯 **Fundamentos teóricos** del aprendizaje supervisado
- 📈 **Algoritmos de clasificación**: Árboles de decisión, SVM, Random Forest, etc.
- 📉 **Algoritmos de regresión**: Regresión lineal, Ridge, Lasso
- 🔬 **Ejemplos prácticos** con datasets reales
- 📊 **Evaluación de modelos**: Métricas y validación
- 💡 **Casos de uso** en industria

---

In [ ]:
# Configuración del entorno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.datasets import make_classification, load_iris, load_wine
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

print('✅ Entorno configurado correctamente')
print(f'NumPy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'Scikit-learn instalado correctamente')

## 1. Fundamentos del Aprendizaje Supervisado

### 1.1 Definición Formal

El aprendizaje supervisado se define como el problema de aproximar una función de mapeo $f: \mathcal{X} \rightarrow \mathcal{Y}$ que relaciona un espacio de entrada $\mathcal{X}$ con un espacio de salida $\mathcal{Y}$.

**Objetivo**: Minimizar el riesgo esperado:

$$R(h) = \mathbb{E}_{(x,y) \sim P(\mathcal{X}, \mathcal{Y})}[\mathcal{L}(h(x), y)]$$

donde:
- $h$ es nuestra hipótesis (modelo)
- $\mathcal{L}$ es la función de pérdida
- $P(\mathcal{X}, \mathcal{Y})$ es la distribución conjunta de los datos

### 1.2 Taxonomía de Problemas

**🎯 Clasificación**: Espacio de salida discreto
- **Binaria**: 2 clases (Spam/No Spam, Fraude/No Fraude)
- **Multiclase**: K > 2 clases (Clasificación de flores, dígitos)
- **Multilabel**: Múltiples etiquetas por instancia

**📈 Regresión**: Espacio de salida continuo
- Predicción de precios
- Estimación de temperatura
- Forecasting de ventas

### 1.3 Flujo de Trabajo Típico

1. **Recolección de datos** etiquetados
2. **Preprocesamiento**: Limpieza, normalización, encoding
3. **División**: Train/Validation/Test (60-20-20 o 70-15-15)
4. **Entrenamiento**: Ajustar parámetros del modelo
5. **Validación**: Ajustar hiperparámetros
6. **Evaluación**: Medir performance en test set
7. **Despliegue**: Poner en producción

In [ ]:
# Ejemplo Práctico: Creación de Dataset Sintético para Clasificación

print("🎲 Generando dataset sintético de clasificación binaria...")
X, y = make_classification(
    n_samples=1000,           # 1000 ejemplos
    n_features=2,             # 2 características (para visualizar)
    n_informative=2,          # Ambas características son informativas
    n_redundant=0,            # Sin características redundantes
    n_clusters_per_class=1,   # 1 cluster por clase
    random_state=42
)

# Crear DataFrame para mejor visualización
df = pd.DataFrame(X, columns=['Feature_1', 'Feature_2'])
df['Target'] = y

print(f"✅ Dataset generado: {df.shape[0]} ejemplos, {df.shape[1]-1} características")
print(f"\nDistribución de clases:")
print(df['Target'].value_counts())

# Visualización del dataset
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot de las dos características
scatter = axes[0].scatter(df['Feature_1'], df['Feature_2'], 
                         c=df['Target'], cmap='coolwarm', 
                         alpha=0.6, edgecolors='k', s=50)
axes[0].set_xlabel('Feature 1', fontsize=12)
axes[0].set_ylabel('Feature 2', fontsize=12)
axes[0].set_title('Dataset de Clasificación Binaria', fontsize=14, fontweight='bold')
axes[0].legend(*scatter.legend_elements(), title="Clase")
axes[0].grid(True, alpha=0.3)

# Distribución de cada característica por clase
for feature in ['Feature_1', 'Feature_2']:
    for clase in [0, 1]:
        data = df[df['Target'] == clase][feature]
        axes[1].hist(data, bins=30, alpha=0.5, label=f'Clase {clase} - {feature}')

axes[1].set_xlabel('Valor', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribuciones de Características', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# División Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 División de datos:")
print(f"   Training set: {X_train.shape[0]} ejemplos ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Test set: {X_test.shape[0]} ejemplos ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\n✅ Datos preparados para entrenamiento")

## 2. Algoritmos de Clasificación

### 2.1 Árboles de Decisión

Los **árboles de decisión** son modelos jerárquicos que particionan el espacio de características mediante reglas if-then.

**Fundamento Matemático**:

**Entropía de Shannon**:
$$H(S) = -\sum_{c \in C} p_c \log_2(p_c)$$

**Índice de Gini**:
$$\text{Gini}(S) = 1 - \sum_{c \in C} p_c^2$$

**Ventajas**:
- ✅ Altamente interpretables
- ✅ No requieren normalización
- ✅ Manejan características numéricas y categóricas
- ✅ Capturan relaciones no lineales

**Desventajas**:
- ❌ Alta varianza (inestables)
- ❌ Tendencia al overfitting
- ❌ Sesgo hacia características con muchos valores

In [ ]:
# Ejemplo Práctico: Árbol de Decisión

from sklearn.tree import DecisionTreeClassifier, plot_tree

print("🌳 Entrenando Árbol de Decisión...")

# Crear y entrenar modelo
dt_model = DecisionTreeClassifier(
    max_depth=4,              # Limitar profundidad para evitar overfitting
    min_samples_split=20,     # Mínimo de muestras para dividir nodo
    min_samples_leaf=10,      # Mínimo de muestras en hojas
    random_state=42
)

dt_model.fit(X_train, y_train)

# Predicciones
y_pred_train = dt_model.predict(X_train)
y_pred_test = dt_model.predict(X_test)

# Evaluación
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

print(f"\n📊 Resultados del Árbol de Decisión:")
print(f"   Training Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"   Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"   Diferencia (overfitting): {(train_acc - test_acc)*100:.2f}%")

# Visualización del árbol
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Visualizar estructura del árbol
plot_tree(dt_model, 
          feature_names=['Feature_1', 'Feature_2'],
          class_names=['Class_0', 'Class_1'],
          filled=True,
          rounded=True,
          ax=axes[0],
          fontsize=10)
axes[0].set_title('Estructura del Árbol de Decisión', fontsize=14, fontweight='bold')

# Frontera de decisión
h = 0.02  # Step size en la malla
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z = dt_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

axes[1].contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
axes[1].scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
               cmap='coolwarm', edgecolors='k', s=50, alpha=0.8)
axes[1].set_xlabel('Feature 1', fontsize=12)
axes[1].set_ylabel('Feature 2', fontsize=12)
axes[1].set_title('Frontera de Decisión (Test Set)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Importancia de características
importances = dt_model.feature_importances_
print(f"\n🔍 Importancia de características:")
for i, imp in enumerate(importances):
    print(f"   Feature_{i+1}: {imp:.4f} ({imp*100:.2f}%)")

### 2.2 Support Vector Machines (SVM)

Las **SVM** buscan el hiperplano óptimo que maximiza el margen entre clases.

**Formulación Matemática**:

Para clasificación binaria linealmente separable:

$$\min_{w,b} \frac{1}{2}\|w\|^2$$
$$\text{sujeto a: } y_i(w^T x_i + b) \geq 1, \forall i$$

**Kernel Trick**: Para problemas no lineales, se mapean datos a espacios de mayor dimensión mediante kernels:

- **Lineal**: $K(x_i, x_j) = x_i^T x_j$
- **RBF (Gaussian)**: $K(x_i, x_j) = \exp(-\gamma \|x_i - x_j\|^2)$
- **Polinomial**: $K(x_i, x_j) = (\gamma x_i^T x_j + r)^d$

**Ventajas**:
- ✅ Efectivo en espacios de alta dimensión
- ✅ Robusto a overfitting (especialmente con regularización)
- ✅ Versatilidad mediante kernels

**Desventajas**:
- ❌ Entrenamiento computacionalmente costoso (O(n²) a O(n³))
- ❌ Difícil interpretabilidad
- ❌ Sensible a escalado de características

In [ ]:
# Ejemplo Práctico: Support Vector Machine

from sklearn.svm import SVC

print("🎯 Entrenando SVM con diferentes kernels...")

# Normalizar datos (CRÍTICO para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelos con diferentes kernels
kernels = ['linear', 'rbf', 'poly']
svm_models = {}
results = {}

for kernel in kernels:
    print(f"\n   Entrenando SVM con kernel {kernel}...")
    
    if kernel == 'poly':
        model = SVC(kernel=kernel, degree=3, C=1.0, random_state=42)
    else:
        model = SVC(kernel=kernel, C=1.0, random_state=42)
    
    model.fit(X_train_scaled, y_train)
    svm_models[kernel] = model
    
    # Evaluación
    train_score = model.score(X_train_scaled, y_train)
    test_score = model.score(X_test_scaled, y_test)
    
    results[kernel] = {'train': train_score, 'test': test_score}
    print(f"      Train Acc: {train_score:.4f} | Test Acc: {test_score:.4f}")

# Visualización comparativa
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, kernel in enumerate(kernels):
    model = svm_models[kernel]
    
    # Crear malla para frontera de decisión
    h = 0.02
    x_min, x_max = X_train_scaled[:, 0].min() - 1, X_train_scaled[:, 0].max() + 1
    y_min, y_max = X_train_scaled[:, 1].min() - 1, X_train_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[idx].contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
    axes[idx].scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], 
                     c=y_test, cmap='coolwarm', edgecolors='k', s=50, alpha=0.8)
    
    # Marcar support vectors
    if hasattr(model, 'support_vectors_'):
        axes[idx].scatter(model.support_vectors_[:, 0], 
                         model.support_vectors_[:, 1],
                         s=100, linewidth=1.5, facecolors='none', 
                         edgecolors='black', label='Support Vectors')
    
    axes[idx].set_xlabel('Feature 1 (scaled)', fontsize=11)
    axes[idx].set_ylabel('Feature 2 (scaled)', fontsize=11)
    axes[idx].set_title(f'SVM - Kernel {kernel.upper()}\nTest Acc: {results[kernel]["test"]:.3f}', 
                       fontsize=12, fontweight='bold')
    axes[idx].legend(loc='upper right')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Mejor modelo
best_kernel = max(results, key=lambda k: results[k]['test'])
print(f"\n🏆 Mejor kernel: {best_kernel} con Test Accuracy = {results[best_kernel]['test']:.4f}")
print(f"   Número de support vectors: {len(svm_models[best_kernel].support_vectors_)}")

### 2.3 Random Forest y Ensemble Methods

**Random Forest** es un método de ensemble que combina múltiples árboles de decisión mediante **bagging** (Bootstrap Aggregating) y **feature randomness**.

**Proceso**:
1. Crear múltiples muestras bootstrap del dataset
2. Para cada muestra, entrenar un árbol con subset aleatorio de características
3. Agregar predicciones mediante votación (clasificación) o promedio (regresión)

**Ventajas**:
- ✅ Reduce overfitting vs árboles individuales
- ✅ Robusto a outliers
- ✅ Mide importancia de características
- ✅ Maneja datasets grandes eficientemente

**Hiperparámetros Clave**:
- `n_estimators`: Número de árboles
- `max_depth`: Profundidad máxima de cada árbol
- `max_features`: Número de características por split
- `min_samples_split`: Mínimo de muestras para dividir nodo

In [ ]:
# Ejemplo Práctico: Random Forest

from sklearn.ensemble import RandomForestClassifier

print("🌲🌲🌲 Entrenando Random Forest...")

# Entrenar modelo
rf_model = RandomForestClassifier(
    n_estimators=100,        # 100 árboles
    max_depth=10,            # Profundidad máxima
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',     # sqrt(n_features) por split
    random_state=42,
    n_jobs=-1                # Usar todos los CPU
)

rf_model.fit(X_train, y_train)

# Evaluación
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)
y_proba_test_rf = rf_model.predict_proba(X_test)[:, 1]

train_acc_rf = accuracy_score(y_train, y_pred_train_rf)
test_acc_rf = accuracy_score(y_test, y_pred_test_rf)

print(f"\n📊 Resultados Random Forest:")
print(f"   Training Accuracy: {train_acc_rf:.4f}")
print(f"   Test Accuracy: {test_acc_rf:.4f}")
print(f"   Diferencia: {(train_acc_rf - test_acc_rf)*100:.2f}%")

# Comparar con árbol individual y SVM
print(f"\n📈 Comparación de Modelos (Test Accuracy):")
print(f"   Árbol de Decisión: {test_acc:.4f}")
print(f"   SVM ({best_kernel}):  {results[best_kernel]['test']:.4f}")
print(f"   Random Forest:     {test_acc_rf:.4f}")

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Frontera de decisión
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z_rf = rf_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z_rf = Z_rf.reshape(xx.shape)

axes[0, 0].contourf(xx, yy, Z_rf, alpha=0.4, cmap='coolwarm')
axes[0, 0].scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
                  cmap='coolwarm', edgecolors='k', s=50, alpha=0.8)
axes[0, 0].set_title(f'Frontera de Decisión Random Forest\nTest Acc: {test_acc_rf:.3f}', 
                    fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Feature 1')
axes[0, 0].set_ylabel('Feature 2')
axes[0, 0].grid(True, alpha=0.3)

# 2. Importancia de características
importances_rf = rf_model.feature_importances_
indices = np.argsort(importances_rf)[::-1]
axes[0, 1].bar(range(len(importances_rf)), importances_rf[indices], color='steelblue')
axes[0, 1].set_xticks(range(len(importances_rf)))
axes[0, 1].set_xticklabels([f'Feature_{i+1}' for i in indices])
axes[0, 1].set_title('Importancia de Características', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Importancia')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba_test_rf)
roc_auc = roc_auc_score(y_test, y_proba_test_rf)

axes[1, 0].plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[1, 0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[1, 0].set_xlim([0.0, 1.0])
axes[1, 0].set_ylim([0.0, 1.05])
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('Curva ROC', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc="lower right")
axes[1, 0].grid(True, alpha=0.3)

# 4. Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1], 
            cbar_kws={'label': 'Número de predicciones'})
axes[1, 1].set_title('Matriz de Confusión', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Clase Real')
axes[1, 1].set_xlabel('Clase Predicha')

plt.tight_layout()
plt.show()

print(f"\n🎯 Métricas detalladas:")
print(classification_report(y_test, y_pred_test_rf, target_names=['Clase 0', 'Clase 1']))

## 3. Algoritmos de Regresión

### 3.1 Regresión Lineal

**Definición**: Modelar relación lineal entre variables predictoras y variable objetivo continua.

**Formulación Matemática**:
$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \epsilon$$

**Función de Costo** (Mean Squared Error):
$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

**Solución Analítica** (Least Squares):
$$\hat{\beta} = (X^TX)^{-1}X^Ty$$

### 3.2 Regresión Ridge (L2 Regularization)

Añade penalización L2 para prevenir overfitting:

$$\text{Loss} = MSE + \alpha \sum_{j=1}^{p}\beta_j^2$$

- $\alpha$: Parámetro de regularización (mayor α → más regularización)

### 3.3 Regresión Lasso (L1 Regularization)

Añade penalización L1, que puede llevar coeficientes a cero (feature selection):

$$\text{Loss} = MSE + \alpha \sum_{j=1}^{p}|\beta_j|$$

**Comparación**:
- **Ridge**: Reduce magnitud de coeficientes, no los elimina
- **Lasso**: Puede eliminar características (sparse solutions)
- **Elastic Net**: Combinación de L1 y L2

In [ ]:
# Ejemplo Práctico: Regresión Lineal vs Ridge vs Lasso

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("📈 Generando dataset para regresión...")

# Generar dataset de regresión
X_reg, y_reg = make_regression(
    n_samples=500,
    n_features=10,
    n_informative=5,
    noise=10,
    random_state=42
)

# Dividir datos
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Normalizar (importante para Ridge/Lasso)
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print(f"Dataset: {X_reg.shape[0]} muestras, {X_reg.shape[1]} características")

# Entrenar modelos
models_reg = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Ridge (α=10.0)': Ridge(alpha=10.0),
    'Lasso (α=0.1)': Lasso(alpha=0.1),
    'Lasso (α=1.0)': Lasso(alpha=1.0)
}

results_reg = {}

print(f"\n🔬 Entrenando y evaluando modelos de regresión...\n")

for name, model in models_reg.items():
    model.fit(X_train_reg_scaled, y_train_reg)
    
    y_pred_train = model.predict(X_train_reg_scaled)
    y_pred_test = model.predict(X_test_reg_scaled)
    
    train_r2 = r2_score(y_train_reg, y_pred_train)
    test_r2 = r2_score(y_test_reg, y_pred_test)
    test_mse = mean_squared_error(y_test_reg, y_pred_test)
    test_mae = mean_absolute_error(y_test_reg, y_pred_test)
    
    # Contar coeficientes no-cero (para Lasso)
    non_zero = np.sum(np.abs(model.coef_) > 1e-5)
    
    results_reg[name] = {
        'train_r2': train_r2,
        'test_r2': test_r2,
        'mse': test_mse,
        'mae': test_mae,
        'non_zero_coef': non_zero,
        'coef': model.coef_
    }
    
    print(f"{name:20s} | R² Train: {train_r2:.4f} | R² Test: {test_r2:.4f} | MSE: {test_mse:.2f} | Features: {non_zero}/{len(model.coef_)}")

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación de R² scores
models_names = list(results_reg.keys())
train_r2_scores = [results_reg[m]['train_r2'] for m in models_names]
test_r2_scores = [results_reg[m]['test_r2'] for m in models_names]

x_pos = np.arange(len(models_names))
width = 0.35

axes[0, 0].bar(x_pos - width/2, train_r2_scores, width, label='Train R²', alpha=0.8, color='steelblue')
axes[0, 0].bar(x_pos + width/2, test_r2_scores, width, label='Test R²', alpha=0.8, color='coral')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('Comparación de Modelos de Regresión', fontsize=13, fontweight='bold')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(models_names, rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')
axes[0, 0].axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='Perfect Score')

# 2. Visualización de coeficientes
for idx, name in enumerate(['Linear Regression', 'Ridge (α=10.0)', 'Lasso (α=1.0)']):
    coefs = results_reg[name]['coef']
    color = ['steelblue', 'orange', 'green'][idx]
    axes[0, 1].plot(range(len(coefs)), coefs, marker='o', label=name, alpha=0.7, linewidth=2)

axes[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0, 1].set_xlabel('Feature Index')
axes[0, 1].set_ylabel('Coefficient Value')
axes[0, 1].set_title('Comparación de Coeficientes', fontsize=13, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Predicciones vs valores reales (mejor modelo)
best_model_name = max(results_reg, key=lambda k: results_reg[k]['test_r2'])
best_model = models_reg[best_model_name]
y_pred_best = best_model.predict(X_test_reg_scaled)

axes[1, 0].scatter(y_test_reg, y_pred_best, alpha=0.6, edgecolors='k', s=50)
axes[1, 0].plot([y_test_reg.min(), y_test_reg.max()], 
               [y_test_reg.min(), y_test_reg.max()], 
               'r--', lw=2, label='Perfect Prediction')
axes[1, 0].set_xlabel('Valores Reales')
axes[1, 0].set_ylabel('Valores Predichos')
axes[1, 0].set_title(f'Predicciones vs Reales - {best_model_name}\nR² = {results_reg[best_model_name]["test_r2"]:.4f}', 
                    fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Residuos
residuals = y_test_reg - y_pred_best
axes[1, 1].scatter(y_pred_best, residuals, alpha=0.6, edgecolors='k', s=50)
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Valores Predichos')
axes[1, 1].set_ylabel('Residuos (Real - Predicho)')
axes[1, 1].set_title(f'Análisis de Residuos - {best_model_name}', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🏆 Mejor modelo: {best_model_name}")
print(f"   Test R²: {results_reg[best_model_name]['test_r2']:.4f}")
print(f"   Test MSE: {results_reg[best_model_name]['mse']:.2f}")
print(f"   Test MAE: {results_reg[best_model_name]['mae']:.2f}")

## 4. Métricas de Evaluación

### 4.1 Métricas para Clasificación

**Matriz de Confusión**:
```
                Predicted
                Neg    Pos
Actual Neg      TN     FP
       Pos      FN     TP
```

**Métricas Derivadas**:

- **Accuracy**: $\frac{TP + TN}{TP + TN + FP + FN}$ 
  - Útil con clases balanceadas

- **Precision**: $\frac{TP}{TP + FP}$
  - "De los predichos positivos, ¿cuántos son correctos?"
  - Alta precision → Pocos falsos positivos

- **Recall (Sensitivity)**: $\frac{TP}{TP + FN}$
  - "De los positivos reales, ¿cuántos detectamos?"
  - Alto recall → Pocos falsos negativos

- **F1-Score**: $2 \times \frac{Precision \times Recall}{Precision + Recall}$
  - Media armónica, balance entre precision y recall

- **ROC-AUC**: Área bajo la curva ROC
  - Mide capacidad de discriminación del modelo
  - 1.0 = Perfecto, 0.5 = Random

**¿Cuándo usar cada métrica?**
- **Accuracy**: Clases balanceadas, costo igual de errores
- **Precision**: Minimizar falsos positivos (ej: spam detection)
- **Recall**: Minimizar falsos negativos (ej: detección de enfermedades)
- **F1**: Balance entre precision y recall
- **ROC-AUC**: Evaluación global, independiente del threshold

### 4.2 Métricas para Regresión

- **MSE (Mean Squared Error)**: $\frac{1}{n}\sum(y_i - \hat{y}_i)^2$
  - Penaliza errores grandes cuadráticamente

- **RMSE (Root MSE)**: $\sqrt{MSE}$
  - Misma unidad que variable objetivo

- **MAE (Mean Absolute Error)**: $\frac{1}{n}\sum|y_i - \hat{y}_i|$
  - Más robusto a outliers que MSE

- **R² (Coefficient of Determination)**: $1 - \frac{SS_{res}}{SS_{tot}}$
  - Proporción de varianza explicada
  - 1.0 = Ajuste perfecto, 0.0 = Modelo constante

- **MAPE (Mean Absolute Percentage Error)**: $\frac{100}{n}\sum\frac{|y_i - \hat{y}_i|}{|y_i|}$
  - Error relativo en porcentaje

## 5. Caso Práctico: Dataset Real (Iris)

Aplicaremos lo aprendido en un dataset real muy conocido: **Iris Dataset**

- 150 muestras de flores
- 4 características: longitud/ancho de sépalo y pétalo
- 3 clases: Setosa, Versicolor, Virginica

Objetivo: Construir un clasificador multiclase óptimo

In [ ]:
# Caso Práctico Completo: Iris Dataset

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

print("🌺 Cargando Iris Dataset...")

# Cargar datos
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Crear DataFrame para exploración
df_iris = pd.DataFrame(X_iris, columns=iris.feature_names)
df_iris['species'] = [iris.target_names[i] for i in y_iris]

print(f"\nDataset: {X_iris.shape[0]} muestras, {X_iris.shape[1]} características")
print(f"\nPrimeras 5 filas:")
print(df_iris.head())

print(f"\nDistribución de clases:")
print(df_iris['species'].value_counts())

# Visualización exploratoria
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Pairplot simplificado
colors = ['red', 'green', 'blue']
for idx, species in enumerate(iris.target_names):
    mask = df_iris['species'] == species
    axes[0].scatter(df_iris[mask]['sepal length (cm)'], 
                   df_iris[mask]['sepal width (cm)'],
                   c=colors[idx], label=species, alpha=0.7, s=80, edgecolors='k')

axes[0].set_xlabel('Sepal Length (cm)', fontsize=11)
axes[0].set_ylabel('Sepal Width (cm)', fontsize=11)
axes[0].set_title('Iris Dataset - Sepal Dimensions', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot de características
df_iris.iloc[:, :4].boxplot(ax=axes[1])
axes[1].set_ylabel('cm', fontsize=11)
axes[1].set_title('Distribución de Características', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# División de datos
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=42, stratify=y_iris
)

# Normalizar
scaler_iris = StandardScaler()
X_train_iris_scaled = scaler_iris.fit_transform(X_train_iris)
X_test_iris_scaled = scaler_iris.transform(X_test_iris)

# Entrenar múltiples modelos
models_iris = {
    'Logistic Regression': LogisticRegression(max_iter=200, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', C=1.0, random_state=42),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

results_iris = {}

print(f"\n🔬 Entrenando y evaluando 6 modelos diferentes...\n")

for name, model in models_iris.items():
    # Entrenar
    if name in ['SVM (RBF)', 'KNN (k=5)', 'Logistic Regression']:
        model.fit(X_train_iris_scaled, y_train_iris)
        y_pred = model.predict(X_test_iris_scaled)
    else:
        model.fit(X_train_iris, y_train_iris)
        y_pred = model.predict(X_test_iris)
    
    # Evaluar
    acc = accuracy_score(y_test_iris, y_pred)
    precision = precision_score(y_test_iris, y_pred, average='weighted')
    recall = recall_score(y_test_iris, y_pred, average='weighted')
    f1 = f1_score(y_test_iris, y_pred, average='weighted')
    
    results_iris[name] = {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': y_pred
    }
    
    print(f"{name:20s} | Acc: {acc:.4f} | Prec: {precision:.4f} | Rec: {recall:.4f} | F1: {f1:.4f}")

# Visualización de resultados
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Comparación de métricas
metrics = ['accuracy', 'precision', 'recall', 'f1']
x_pos = np.arange(len(models_iris))
width = 0.2

for idx, metric in enumerate(metrics):
    values = [results_iris[m][metric] for m in models_iris.keys()]
    axes[0].bar(x_pos + idx*width, values, width, label=metric.capitalize(), alpha=0.8)

axes[0].set_ylabel('Score')
axes[0].set_title('Comparación de Modelos en Iris Dataset', fontsize=13, fontweight='bold')
axes[0].set_xticks(x_pos + width * 1.5)
axes[0].set_xticklabels(models_iris.keys(), rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0.85, 1.01])

# Matriz de confusión del mejor modelo
best_model_iris = max(results_iris, key=lambda k: results_iris[k]['accuracy'])
cm_iris = confusion_matrix(y_test_iris, results_iris[best_model_iris]['predictions'])

sns.heatmap(cm_iris, annot=True, fmt='d', cmap='YlGnBu', ax=axes[1],
            xticklabels=iris.target_names, yticklabels=iris.target_names)
axes[1].set_title(f'Matriz de Confusión - {best_model_iris}\nAccuracy: {results_iris[best_model_iris]["accuracy"]:.4f}', 
                 fontsize=13, fontweight='bold')
axes[1].set_ylabel('Clase Real')
axes[1].set_xlabel('Clase Predicha')

plt.tight_layout()
plt.show()

print(f"\n🏆 Mejor modelo: {best_model_iris}")
print(f"   Accuracy: {results_iris[best_model_iris]['accuracy']:.4f}")
print(f"\n📊 Classification Report:")
print(classification_report(y_test_iris, results_iris[best_model_iris]['predictions'], 
                           target_names=iris.target_names))

## 6. Conclusiones y Mejores Prácticas

### 📚 Resumen de Conceptos Clave

1. **Aprendizaje Supervisado**: Paradigma fundamental del ML que utiliza datos etiquetados para entrenar modelos predictivos

2. **Algoritmos Principales**:
   - **Árboles de Decisión**: Interpretables, útiles para problemas de clasificación y regresión
   - **SVM**: Potentes para problemas no lineales mediante kernels
   - **Random Forest**: Robustos mediante ensambles de árboles
   - **Regresión**: Múltiples variantes (Linear, Ridge, Lasso) para problemas de predicción numérica

3. **Métricas de Evaluación**:
   - **Clasificación**: Accuracy, Precision, Recall, F1-Score, ROC-AUC
   - **Regresión**: MSE, RMSE, MAE, R², MAPE

### ✅ Mejores Prácticas

- **Preprocesamiento**: Normalizar/escalar datos, especialmente para SVM y modelos basados en distancia
- **División de Datos**: Usar train/test split (70-30 o 80-20) y validación cruzada
- **Selección de Modelos**: Comparar múltiples algoritmos antes de seleccionar el mejor
- **Hiperparámetros**: Optimizar mediante GridSearch o RandomSearch
- **Validación**: Siempre evaluar en datos no vistos (test set)
- **Interpretabilidad vs Performance**: Balance entre explicabilidad y precisión

### 🎯 Ejercicios Propuestos

1. **Ejercicio 1**: Implementa un clasificador de dígitos manuscritos usando el dataset MNIST con Random Forest
2. **Ejercicio 2**: Compara el rendimiento de diferentes kernels de SVM en un problema de clasificación binaria
3. **Ejercicio 3**: Crea un modelo de regresión para predecir precios de casas (Boston Housing dataset)
4. **Ejercicio 4**: Implementa validación cruzada k-fold y compara con train/test split simple
5. **Ejercicio 5**: Optimiza hiperparámetros de un modelo usando GridSearchCV

### 📖 Recursos Adicionales

- **Scikit-learn Documentation**: https://scikit-learn.org/stable/supervised_learning.html
- **Libro**: "Introduction to Statistical Learning" by James et al.
- **Curso**: "Machine Learning" de Andrew Ng (Coursera)
- **Kaggle Competitions**: Práctica con problemas reales de ML supervisado

---

**🎓 Siguiente Notebook**: [ML No Supervisado](../02_Ml_no_supervisado/ml_no_supervisado.ipynb)

En el siguiente módulo exploraremos técnicas de aprendizaje no supervisado, incluyendo clustering, reducción de dimensionalidad y detección de anomalías.